In [215]:
import sys 
sys.path.insert(0, '../')

In [247]:
from torch.utils.data import DataLoader
import pandas as pd
from common.common import create_folder,H5Recorder
from common.conf_loader import ConfLoader
import configs.config_classes as conf_objs
from configs.config_classes import BertConfig
import numpy as np
from torch.utils.data.dataset import Dataset
import os
import torch
import torch.nn as nn
import pytorch_pretrained_bert as Bert

from model import optimiser
import sklearn.metrics as skm
import math
from torch.utils.data.dataset import Dataset
import random
import numpy as np
import torch
import time
from sklearn.metrics import roc_auc_score
from common.common import load_obj
from model.utils import age_vocab
from dataLoader.NextXVisit import NextVisit
from model.NextXVisit import BertForMultiLabelPrediction
import warnings
import random
warnings.filterwarnings(action='ignore')


conf_loader = ConfLoader('../configs/proact')

In [248]:
# seeds
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

In [249]:
temp = pd.read_parquet(f'../data/for_behrt_wrt_months_more_than_3_months_fixed.parquet')#.iloc[:2536,:]

In [250]:
temp = temp.rename(columns={'ID':'PID', 'FRS_diag':'data'})
temp.head()

,PID,data,rank,age,ALSFRS_Total,time,FVC,CK,Creatinine,Phosphorus,...,Potassium,Protein,Glucose,Calcium,Sodium,Platelets,fixed_time,fixed_months,3_months_idx,FRS_diag_original
0,329,"[CLS, Q1_Speech_score_4, Q2_Salivation_score_3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 38, 3...","[27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 27, 2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 42, 42, 4...","[0.5294117647058824, 0.5294117647058824, 0.529...","[0.5271317829457365, 0.5271317829457365, 0.527...","[0.5005656108597285, 0.5005656108597285, 0.500...","[0.3333333333333326, 0.3333333333333326, 0.333...",...,"[-0.2499999999999989, -0.2499999999999989, -0....","[-0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0....","[-0.3277272727272726, -0.3277272727272726, -0....","[-0.484466019417476, -0.484466019417476, -0.48...","[-0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0.5, -0....","[0.49333333333333335, 0.49333333333333335, 0.4...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 42, 42, 4...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",33,"[CLS, Q1_Speech_score_4, Q2_Salivation_score_3..."
1,533,"[CLS, Q1_Speech_score_1, Q2_Salivation_score_2...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65, 65]","[30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30, 30]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.029411764705882353, 0.029411764705882353, 0...","[-0.37209302325581395, -0.37209302325581395, -...","[0.25056561085972845, 0.25056561085972845, 0.2...","[-2.0000000000000027, -2.0000000000000027, -2....",...,"[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...","[-0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0.4, -0....","[1.9427272727272722, 1.9427272727272722, 1.942...","[-0.6459546925566347, -0.6459546925566347, -0....","[0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.75, 0.7...","[-0.8933333333333333, -0.8933333333333333, -0....","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",11,"[CLS, Q1_Speech_score_1, Q2_Salivation_score_2..."
2,649,"[CLS, Q1_Speech_score_3, Q2_Salivation_score_3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 48, 4...","[36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 32, 3...","[1.1764705882352942, 1.1764705882352942, 1.176...","[-0.5697674418604651, -0.5697674418604651, -0....","[-0.4994343891402715, -0.4994343891402715, -0....","[-1.3333333333333355, -1.3333333333333355, -1....",...,"[0.2500000000000011, 0.2500000000000011, 0.250...","[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, ...","[-0.42863636363636326, -0.42863636363636326, -...","[-0.9689320388349492, -0.9689320388349492, -0....","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.38666666666666666, 0.38666666666666666, 0.3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 32, 32, 3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",44,"[CLS, Q1_Speech_score_3, Q2_Salivation_score_3..."
3,708,"[CLS, Q1_Speech_score_3, Q2_Salivation_score_3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, ...","[46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 46, 4...","[33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 33, 3...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 35, 3...","[-0.8235294117647058, -0.8235294117647058, -0....","[-0.5271317829457365, -0.5271317829457365, -0....","[0.5005656108597285, 0.5005656108597285, 0.500...","[-1.3333333333333355, -1.3333333333333355, -1....",...,"[2.249999999999999, 2.249999999999999, 2.24999...","[1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, 1.6, ...","[-0.3277272727272726, -0.3277272727272726, -0....","[1.1304207119741108, 1.1304207119741108, 1.130...","[1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.25, 1.2...","[0.013333333333333334, 0.013333333333333334, 0...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 35, 35, 3...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",33,"[CLS, Q1_Speech_score_3, Q2_Salivation_score_3..."
4,1234,"[CLS, Q1_Speech_score_2, Q2_Salivation_score_3...","[0, 0, 0,

In [251]:
lab_test = 'creatinine'
# preper target
def get_days(months):
    return (months / 12) * 365.24
dt = 12
valid_for_train_ub =  get_days(3 + dt) + 7
valid_for_train_lb =  get_days(3 + dt) - 7
def get_last_valid_idx(ls):
    s = pd.Series([valid_for_train_lb <= i <= valid_for_train_ub  for i in ls])
    return s.where(s).last_valid_index()
temp['target_idx'] = temp['fixed_time'].apply(get_last_valid_idx)

train = temp.iloc[:2536,:].reset_index()
test = temp.iloc[2536:, :].reset_index()



In [252]:
train=train.loc[~train['target_idx'].isna(),:].reset_index()
test=test.loc[~test['target_idx'].isna(),:].reset_index()
train['target_idx']=train['target_idx'].astype(int)
test['target_idx']=test['target_idx'].astype(int)
# temp.apply(lambda x: x['FRS_diag_original'][:x['target_idx']+1], axis=1)

In [253]:
train['target'] = train.apply(lambda x: x['FRS_diag_original'][x['target_idx']-10:x['target_idx']], axis=1)
test['target'] = test.apply(lambda x: x['FRS_diag_original'][x['target_idx']-10:x['target_idx']], axis=1)


In [254]:

train_s = train.iloc[:int(train.shape[0]*.8),].reset_index(drop=True)
validation = train.iloc[int(train.shape[0]*.8):,].reset_index(drop=True)

In [255]:
n_train = train_s.shape[0]
n_validation = validation.shape[0]
n_test = test.shape[0]

In [256]:
BEST_MODEL_NAME = f'best_als_visit_next_{dt}_months_predictor_{lab_test}_small'
file_config = conf_loader.load_configs('file_config_next_visit.json')
global_params = conf_loader.load_configs('global_params.json')
optim_config = conf_loader.load_configs('optimizer_params.json')
file_config['saved_model_name'] = BEST_MODEL_NAME
pretrain_model_path = 'model_path/proact_model_visits_creatinine_small"'

In [257]:

train.to_parquet(file_config.get('train_all'))
train_s.to_parquet(file_config.get('train'))
validation.to_parquet(file_config.get('validation'))
test.to_parquet(file_config.get('test'))


In [258]:
BertVocab = load_obj(file_config['vocab'])
ageVocab, _ = age_vocab(max_age=global_params['max_age'], symbol=global_params['age_symbol'])

In [259]:
ageVocab = {k:v for k,v in ageVocab.items() if v <=85 }


In [260]:
# re-format label token
def format_label_vocab(token2idx):
    token2idx = token2idx.copy()
    del token2idx['PAD']
    del token2idx['SEP']
    del token2idx['CLS']
    del token2idx['MASK']
    token = list(token2idx.keys())
    labelVocab = {}
    for i,x in enumerate(token):
        labelVocab[x] = i
    return labelVocab

labelVocab = format_label_vocab(BertVocab['token2idx'])

In [261]:
model_config = conf_loader.load_model_configs('model_params.json', global_params, BertVocab, ageVocab)

feature_dict = {
    'word':True,
    'seg':True,
    'age':True,
    'position': True,
    'creatinine':True,
}


In [262]:
def get_loader(df, num_workers=3):
    Dset = NextVisit(token2idx=BertVocab['token2idx'], label2idx=labelVocab, age2idx=ageVocab, dataframe=df, max_len=global_params['max_len_seq'],
                 code='data', label='target', patid='PID')
    return DataLoader(dataset=Dset, batch_size=global_params['batch_size'], shuffle=True, num_workers=num_workers)


train = pd.read_parquet(file_config['train'])
test = pd.read_parquet(file_config['test'])

train_load = get_loader(train)
test_load = get_loader(test)



In [263]:
# del model
conf = BertConfig(model_config)
model = BertForMultiLabelPrediction(conf, num_labels=len(labelVocab.keys()), feature_dict=feature_dict)

In [264]:
def load_model(path, model):
    # load pretrained model and update weights
    pretrained_dict = torch.load(path)
    model_dict = model.state_dict()
    # 1. filter out unnecessary keys
    pretrained_dict = {k: v for k, v in pretrained_dict.items() if k in model_dict}
    # 2. overwrite entries in the existing state dict
    model_dict.update(pretrained_dict)
    # 3. load the new state dict
    model.load_state_dict(model_dict)
    return model



In [265]:
model = model.to(global_params['device'])
optim = optimiser.adam(params=list(model.named_parameters()), config=optim_config)

t_total value of -1 results in schedule not being applied


In [266]:
def train(e):
    model.train()
    tr_loss = 0
    temp_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    cnt = 0
    for step, batch in enumerate(train_load):
        cnt +=1
        age_ids, input_ids, posi_ids, creatinine, segment_ids, attMask, targets, _ = batch

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine,attention_mask=attMask, labels=targets)
        
        if global_params['gradient_accumulation_steps'] >1:
            loss = loss/global_params['gradient_accumulation_steps']
        loss.backward()
        
        temp_loss += loss.item()
        tr_loss += loss.item()
        nb_tr_examples += input_ids.size(0)
        nb_tr_steps += 1
        
        if step % 50==0:
            print("epoch: {}\t| Cnt: {}\t| Loss: {}\t".format(e, cnt,temp_loss/50))
            temp_loss = 0
        
        if (step + 1) % global_params['gradient_accumulation_steps'] == 0:
            optim.step()
            optim.zero_grad()

def evaluation():
    model.eval()
    y = []
    y_label = []
    tr_loss = 0
    for step, batch in enumerate(test_load):
        model.eval()
        age_ids, input_ids, posi_ids, creatinine, segment_ids, attMask, targets, _ = batch
        
        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])
        
        with torch.no_grad():
            loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine,attention_mask=attMask, labels=targets)
        logits = [logit.cpu() for logit in logits]
        targets = targets.cpu()
        
        tr_loss += loss.item()

        y_label.append(targets)
        y.append(logits)

    # y_label = torch.cat(y_label, dim=0)
    # y = torch.cat(y, dim=0)

    # aps, roc, output, label = precision_test(y, y_label)
    return tr_loss

In [267]:
best_pre = np.inf
for e in range(100):
    train(e)
    test_loss = evaluation()
    if test_loss < best_pre:
        # Save a trained model
        print("** ** * Saving fine - tuned model ** ** * ")
        model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self
        output_model_file = os.path.join(file_config['output_dir'],file_config['saved_model_name'])
        create_folder(file_config['output_dir'])

        torch.save(model_to_save.state_dict(), output_model_file)
        best_pre = test_loss
    print(f' | test loss : {test_loss} ')

epoch: 0	| Cnt: 1	| Loss: 0.013345861434936523	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.6359679698944092 
epoch: 1	| Cnt: 1	| Loss: 0.011321872472763062	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.5789742469787598 
epoch: 2	| Cnt: 1	| Loss: 0.010647249221801759	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.547497034072876 
epoch: 3	| Cnt: 1	| Loss: 0.010360623598098756	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.5335565209388733 
epoch: 4	| Cnt: 1	| Loss: 0.008559824228286743	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.5291281938552856 
epoch: 5	| Cnt: 1	| Loss: 0.008054086565971374	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.5273545384407043 
epoch: 6	| Cnt: 1	| Loss: 0.006739607453346253	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.5262475609779358 
epoch: 7	| Cnt: 1	| Loss: 0.009805093407630921	
** ** * Saving fine - tuned model ** ** * 
 | test loss : 0.52548480033

In [268]:
model = load_model(os.path.join(file_config['output_dir'],file_config['saved_model_name']), model)

In [269]:
test_r = pd.read_parquet(file_config.get('test'))
# test_r = test_r.rename(columns={'FRS_diag':'code', 'target':'label', 'PID':'patid'})

testload = get_loader(test_r)

# Dset = NextVisit(token2idx=BertVocab['token2idx'], label2idx=labelVocab, age2idx=ageVocab, dataframe=test_r, max_len=global_params['max_len_seq'],
#                                  code='data', label='target')
# testload = DataLoader(dataset=Dset, batch_size=global_params['batch_size'], shuffle=False, num_workers=3)


In [270]:

def evaluation():
    model.eval()
    y = []
    y_label = []
    tr_loss = 0
    for step, batch in enumerate(test_load):
        model.eval()
        age_ids, input_ids, posi_ids, creatinine, segment_ids, attMask, targets, _ = batch
        # targets = torch.tensor(mlb.transform(targets.numpy()), dtype=torch.float32)

        age_ids = age_ids.to(global_params['device'])
        input_ids = input_ids.to(global_params['device'])
        posi_ids = posi_ids.to(global_params['device'])
        creatinine = creatinine.to(global_params['device'])
        segment_ids = segment_ids.to(global_params['device'])
        attMask = attMask.to(global_params['device'])
        targets = targets.to(global_params['device'])

        with torch.no_grad():
            loss, logits = model(input_ids, age_ids, segment_ids, posi_ids,creatinine,attention_mask=attMask, labels=targets)
        logits = [logit.cpu() for logit in logits]
        targets = targets.cpu()

        y_label.append(targets)
        y.append(logits)

    y_label = torch.cat(y_label, dim=0)
    # y = torch.cat(y, dim=0)

    # aps, roc, output, label = precision_test(y, y_label)
    return y, y_label



y, y_label = evaluation()
# aps, roc, output, label = precision_test(y, y_label)

In [271]:
y_label
__y =  [torch.cat(_y, dim=1) for _y in y]
__y =  torch.cat(__y)
__y.shape

torch.Size([8, 10])

In [272]:
# (__y * 4 - y_label * 4).abs().mean()

# print(((__y * 4).round() - y_label * 4).abs().mean())
#
print(((__y * 4).round() - y_label * 4).abs().mean(dim=0))
print(((__y * 4) - y_label * 4).abs().mean(dim=0))

a = (__y * 4).round() - y_label * 4
a = a ** 2
a = a.mean(dim=0)
a = a.sqrt()
print(a)
a = (__y * 4) - y_label * 4
a = a ** 2
a = a.mean(dim=0)
data = a.sqrt()


tensor([1.3750, 1.0000, 0.5000, 1.2500, 1.2500, 1.3750, 0.8750, 0.8750, 0.8750,
        0.6250])
tensor([1.3125, 1.0722, 0.6407, 1.1717, 1.2624, 1.4263, 0.8561, 0.7299, 0.8628,
        0.7152])
tensor([1.6202, 1.2247, 0.8660, 1.4142, 1.5000, 1.5411, 1.1726, 1.0607, 1.0607,
        0.9354])


In [273]:
a = (__y * 4).round().sum(dim=1) - (y_label * 4).sum(dim=1)
a = a.abs().mean()
print(a.numpy())

b = (__y * 4).sum(dim=1) - (y_label * 4).sum(dim=1)
b = b.abs().mean()
print(b.numpy())

c = (__y * 4).sum(dim=1) - (y_label * 4).sum(dim=1)
c = (c ** 2).mean().sqrt()
print(c.numpy())

d = (__y * 4).round().sum(dim=1) - (y_label * 4).sum(dim=1)
d = (d **2).mean().sqrt()
print(d.numpy())

8.5
8.002806
8.72438
9.246621


In [274]:
cols=['Speech', 'Salivation',
       'Swallowing', 'Handwriting',
       'Cutting', 'Dressing_and_Hygiene',
       'Turning_in_Bed', 'Walking',
       'Climbing_Stairs', 'Respiratory']
res = pd.DataFrame(data=data.view(1,-1).numpy(), columns=cols)
mae  = (__y * 4).round().sum(dim=1) - (y_label * 4).sum(dim=1).numpy()
res['MAE']=c.numpy()

res['n_train'] = n_train
res['n_validation'] = n_validation
res['n_test'] = n_test
res.to_csv(f'next{dt}months_creatinine_small.csv')

In [244]:
res

,Speech,Salivation,Swallowing,Handwriting,Cutting,Dressing_and_Hygiene,Turning_in_Bed,Walking,Climbing_Stairs,Respiratory,MAE,n_train,n_validation,n_test
0,0.840711,0.808245,0.986925,1.052595,1.081831,0.833618,0.884761,0.785584,0.759341,0.7131,4.785634,220,55,85


In [275]:
print(f'Done {dt} months')

Done 12 months


In [156]:
"""
# (__y * 4 - y_label * 4).abs().mean()

# print(((__y * 4).round() - y_label * 4).abs().mean())
#
print(((__y * 4).round() - y_label * 4).abs().mean(dim=0))
print(((__y * 4) - y_label * 4).abs().mean(dim=0))

tensor([0.3140, 0.5601, 0.3915, 0.4981, 0.4903, 0.4690, 0.5891, 0.4516, 0.5155,
        0.5291])
tensor([0.4384, 0.5715, 0.5213, 0.5532, 0.5293, 0.5035, 0.5767, 0.4699, 0.5478,
        0.5798])

a = (__y * 4).round().sum(dim=1) - (y_label * 4).sum(dim=1)
a = a.abs().mean()
print(a)

b = (__y * 4).sum(dim=1) - (y_label * 4).sum(dim=1)
b = b.abs().mean()
print(b)
tensor(2.6182)
tensor(2.4791)
"""



'\n# (__y * 4 - y_label * 4).abs().mean()\n\n# print(((__y * 4).round() - y_label * 4).abs().mean())\n#\nprint(((__y * 4).round() - y_label * 4).abs().mean(dim=0))\nprint(((__y * 4) - y_label * 4).abs().mean(dim=0))\n\ntensor([0.3140, 0.5601, 0.3915, 0.4981, 0.4903, 0.4690, 0.5891, 0.4516, 0.5155,\n        0.5291])\ntensor([0.4384, 0.5715, 0.5213, 0.5532, 0.5293, 0.5035, 0.5767, 0.4699, 0.5478,\n        0.5798])\n\na = (__y * 4).round().sum(dim=1) - (y_label * 4).sum(dim=1)\na = a.abs().mean()\nprint(a)\n\nb = (__y * 4).sum(dim=1) - (y_label * 4).sum(dim=1)\nb = b.abs().mean()\nprint(b)\ntensor(2.6182)\ntensor(2.4791)\n'

In [287]:
# df = pd.concat([pd.read_csv(f'next{m}months_creatinine.csv') for m in (3,6,12)])
# df = df.rename(columns={df.columns[0]:'nextXmonths'})
# df['nextXmonths'] = (3,6,12)
# df['lab_results'] = ['None']*3
# df = df.set_index('lab_results')
# df.to_csv('results.csv')

[WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next12months.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next12months_creatinine.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next12months_creatinine_small.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next3months.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next3months_creatinine.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next3months_creatinine_small.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next6months.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next6months_creatinine.csv'),
 WindowsPath('C:/Users/mannh/Documents/Git/BEHRT_MSC/task/next6months_creatinine_small.csv')]

In [288]:
from pathlib import Path
t_df = pd.DataFrame()
for file in Path(os.getcwd()).rglob('next*months*.csv'):
    t_df = pd.concat([t_df, pd.read_csv(file).assign(file=file.name.replace('.csv','')).iloc[:,1:]])
t_df = t_df.set_index('file')
t_df

,Speech,Salivation,Swallowing,Handwriting,Cutting,Dressing_and_Hygiene,Turning_in_Bed,Walking,Climbing_Stairs,Respiratory,MAE,n_train,n_validation,n_test
file,,,,,,,,,,,,,,
next12months,1.569659,1.094108,0.611420,1.370049,1.313293,1.572980,0.899035,0.782292,1.162184,1.031419,7.410675,60,15,8
next12months_creatinine,1.667354,1.215627,0.685642,1.299823,1.004725,1.379895,0.609507,0.670426,0.988132,1.007476,6.612772,60,15,8
next12months_creatinine_small,1.623852,1.323368,0.744738,1.352167,1.464452,1.626652,0.975365,0.854922,1.012737,0.960745,8.724380,60,15,8
next3months,0.587059,0.666650,0.664830,0.864030,0.770703,0.698961,0.768706,0.611158,0.726082,0.751973,3.813820,779,195,234
next3months_creatinine,0.631635,0.706722,0.680364,0.853704,0.760213,0.695057,0.770713,0.600466,0.721354,0.751448,3.790790,779,195,234
next3months_creatinine_small,0.616508,0.690694,0.681805,0.850846,0.763858,0.696371,0.786722,0.600480,0.736056,0.786579,3.879244,779,195,234
next6months,0.775040,0.759797,0.940897,1.088254,1.087134,0.851184,0.873855,0.749641,0.775839,0.680189,4.720744,779,195,234
next6months_creatinine,0.787176,0.814266,0.955292,1.049367,1.062354,0.851369,0.897804,0.782094,0.776430,0.662205,4.644844,779,195,234
next6months_creatinine_small,0.840711,0.808245,0.986924,1.052595,1.081831,0.833618,0.884761,0.785584,0.759341,0.713100,4.785634,220,55,85


In [289]:
t_df.to_csv('models_summary.csv')